# Final Project – Machine Learning Applications 
## Julien VOLLET

## Introduction

L’analyse automatique de documents textuels est devenue un enjeu central dans de nombreux domaines tels que la veille d’information, la recommandation personnalisée, ou encore l’analyse d’opinion. Dans ce contexte, le projet que nous avons mené s’inscrit dans le cadre du cours "Applications de l’apprentissage automatique", et a pour objectif de mettre en œuvre une chaîne complète de traitement de données textuelles, depuis le prétraitement jusqu’à la visualisation interactive des résultats.

Nous avons choisi de travailler sur un jeu de données issu de TripAdvisor, centré sur des restaurants situés en France. Ce jeu de données contient des informations textuelles descriptives associées à chaque établissement (comme des mots-clés, types de cuisine, ambiance, etc.), ainsi que des variables numériques telles que la note moyenne ou le nombre d’avis.

Le projet se structure autour de plusieurs tâches successives :

- **Task 1** : Traitement des données textuelles (`keywords`) et vectorisation à l’aide de plusieurs techniques (TF-IDF, Word2Vec, etc.), ainsi qu’une modélisation thématique basée sur LDA.
- **Task 2.1** : Construction d’un modèle de classification supervisée visant à prédire si un restaurant est bien noté à partir de ses descripteurs textuels.
- **Task 3** : Développement d’un tableau de bord interactif avec Dash permettant d’explorer les thèmes extraits automatiquement et leur distribution géographique.

Au travers de ce projet, nous avons appliqué de manière concrète les concepts vus en cours : nettoyage de données textuelles, vectorisation, sélection de caractéristiques, modélisation supervisée et non supervisée, visualisation dynamique et évaluation rigoureuse des performances. Ce rapport retrace l’ensemble de cette démarche, ainsi que les choix techniques et méthodologiques que nous avons faits tout au long du processus.


## 1. Task 1 – Traitement du langage naturel, vectorisation et modélisation thématique

### 1.1 Description du jeu de données

Le jeu de données utilisé provient d’un extrait public du site TripAdvisor, contenant des informations sur plus de 10 000 restaurants en Europe. Pour cadrer notre projet, nous avons filtré les données pour ne conserver que les restaurants situés en **France**.

Les variables retenues après nettoyage sont :

- `restaurant_name` : nom du restaurant  
- `city`, `address` : localisation  
- `latitude`, `longitude` : coordonnées géographiques  
- `cuisines`, `special_diets`, `features`, `atmosphere` : catégories textuelles  
- `avg_rating` : note moyenne (0 à 5)  
- `total_reviews_count` : nombre d’avis  
- `keywords` : mots-clés qualitatifs associés au restaurant  

Après suppression des doublons et des lignes incomplètes, le dataset final contient environ **X restaurants uniques** répartis sur **Y villes françaises**. La variable `keywords`, qui est textuelle, a été utilisée comme base principale pour la vectorisation et l’analyse thématique.

**Illustration :**  
Un exemple de mots-clés pour un restaurant :
> `["vegetarian friendly", "romantic", "gluten free options", "terrace", "wine bar"]`

Nous avons observé une grande diversité sémantique dans ces descripteurs, rendant nécessaire un pipeline de traitement rigoureux.

---

### 1.2 Pipeline de prétraitement des textes

Le prétraitement s’est concentré sur la colonne `keywords`. Celle-ci contient des chaînes textuelles représentant des attributs qualitatifs. L’objectif du nettoyage est d’uniformiser et de simplifier ces données avant leur vectorisation.

Les étapes appliquées sont :

- **Passage en minuscules** : uniformisation des termes  
- **Suppression de la ponctuation** : nettoyage des caractères spéciaux  
- **Suppression de stopwords personnalisés** : mots très fréquents et peu discriminants comme "the", "in", "with"  
- **Tokenisation** : découpage des mots-clés en tokens via un `split()` simple

**Exemple (avant/après) :**

| Texte original                                      | Après nettoyage                    |
|-----------------------------------------------------|-------------------------------------|
| `"Gluten Free Options, Good for Groups, Cozy"`      | `"gluten free options good groups cozy"` |

Chaque restaurant est ainsi représenté par une liste de **tokens courts et spécifiques**, utilisés comme base pour la vectorisation.

Nous avons également analysé la distribution du nombre de tokens (mots-clés nettoyés) par restaurant, ce qui permet de valider l’homogénéité et la densité de l’information textuelle disponible.

![Distribution de tokens](rapport/nb_token.png)

> *Figure : Histogramme du nombre de mots-clés (tokens) par restaurant après nettoyage. La majorité des établissements possèdent entre 6 et 9 tokens.*


Cette distribution montre une **concentration nette autour de 6 à 9 mots-clés par restaurant**, ce qui reflète une certaine standardisation dans la manière dont les établissements sont décrits. Très peu de restaurants en possèdent moins de 5 ou plus de 10, ce qui indique une **densité d’information raisonnablement constante** entre les observations.

Cette régularité est un atout pour l’apprentissage automatique : elle garantit que la plupart des entrées disposent d’un volume suffisant de descripteurs pour permettre une vectorisation stable et une interprétation fiable. Elle justifie également notre choix de recourir à des méthodes comme TF-IDF ou LDA, qui supposent un minimum de contenu lexical pour produire des résultats pertinents.





#### Choix de **ne pas utiliser de lemmatisation**

Nous avons délibérément exclu la lemmatisation pour les raisons suivantes :

- Les `keywords` sont des **phrases fixes** (ex. : "wine bar", "family friendly"), et leur modification pourrait altérer le sens.
- Les variantes morphologiques sont **rares** dans ce contexte (pas de "run/running/ran").
- Des expressions apparemment proches comme `"drinks available"` et `"drink bar"` ont des sens différents — la lemmatisation risquerait de **fusionner des entités non équivalentes**.

Nous avons donc choisi une approche **minimaliste mais sémantiquement sûre** pour conserver la richesse originale des mots-clés.

---

### 1.3 Méthodes de vectorisation explorées

Nous avons testé plusieurs techniques de vectorisation afin de capturer la diversité sémantique des mots-clés. Ces méthodes permettent de transformer les documents en vecteurs numériques exploitables par des modèles de machine learning.

### 1.3 Méthodes de vectorisation explorées

Afin de transformer les données textuelles en représentations numériques exploitables par des algorithmes de machine learning, nous avons testé plusieurs techniques de vectorisation adaptées aux textes courts et spécialisés que sont les `keywords`.

####  TF-IDF (Term Frequency – Inverse Document Frequency)

Le TF-IDF est une méthode pondérée qui attribue à chaque mot une importance proportionnelle à sa fréquence dans un document, mais inversement proportionnelle à sa fréquence dans l’ensemble du corpus. Autrement dit, les mots spécifiques à un restaurant seront mieux valorisés que les mots trop fréquents. Nous avons utilisé l’implémentation `TfidfVectorizer` de Scikit-learn, qui produit une matrice creuse de grande dimension (~5000 tokens uniques). Cette méthode s’est avérée rapide, stable et bien adaptée à notre contexte, car elle ne nécessite pas d’apprentissage préalable et conserve une bonne capacité discriminante.

####  Bag of Words (BoW)

Le modèle BoW repose sur un principe simple : compter combien de fois chaque mot apparaît dans un document, sans prendre en compte ni le contexte ni l’ordre des mots. Sa mise en œuvre avec `CountVectorizer` en fait une méthode très rapide, mais souvent trop naïve. Elle ne distingue pas les mots fréquents pertinents de ceux qui sont banals, et produit une représentation qui peut être bruitée si le vocabulaire est trop générique. Cela dit, elle reste utile pour des modèles simples comme Naive Bayes.

####  Word2Vec

Word2Vec est une méthode de plongement lexical, introduite par Google, qui encode chaque mot sous forme d’un vecteur dense appris par le contexte dans lequel il apparaît. Dans notre projet, nous avons entraîné un modèle Word2Vec avec Gensim. Chaque mot est représenté par un vecteur dense de dimension 100, et un document est représenté par la moyenne des vecteurs de ses mots. Bien que plus riche sémantiquement, cette méthode nécessite un apprentissage préalable et devient moins efficace si le corpus est limité ou peu varié, comme c’est le cas ici.

####  Doc2Vec

Doc2Vec est une extension de Word2Vec qui apprend directement un vecteur dense propre à chaque document, en plus des vecteurs mots. Cela permet de capturer la structure globale d’un texte. Chaque restaurant est associé à un `TaggedDocument`. Doc2Vec est plus adapté à des documents longs ou complexes, mais dans notre cas, les gains obtenus n’ont pas justifié le coût de calcul et la complexité supplémentaire.

---



### 1.4 Modélisation thématique avec LDA

Nous avons exploré la détection automatique de **sujets latents** (topics) dans les documents via l’algorithme **Latent Dirichlet Allocation (LDA)**.

#### 1.4.1 LDA avec Gensim

- Création d’un dictionnaire et d’un corpus bag-of-words.
- Entraînement de plusieurs modèles LDA avec **2 à 20 sujets**.
- Évaluation via le **score de cohérence `c_v`**.
- Le score optimal a été atteint pour **K = X** sujets.

**Graphique** : Score de cohérence en fonction du nombre de topics

#### 1.4.2 LDA avec Scikit-learn

Nous avons également implémenté une version de LDA à l’aide de Scikit-learn. Les `keywords` ont été vectorisés via `CountVectorizer`, puis un modèle `LatentDirichletAllocation` a été entraîné avec **K = 6** sujets, déterminé comme optimal après expérimentation. Chaque restaurant a été associé à un **topic dominant**, à partir de la distribution de probabilité générée.

Pour chaque topic, nous avons extrait les **10 mots-clés les plus représentatifs** et identifié un **thème général** interprétable :

| Topic | Thème général                   | Mots-clés dominants                                                |
|-------|----------------------------------|---------------------------------------------------------------------|
| 0     | **Vin et ambiance romantique**   | `"wine", "romantic", "cozy", "fine dining", "bar", "terrace"`      |
| 1     | **Restauration rapide / urbaine**| `"cheap", "fast", "burger", "takeaway", "quick", "snack"`          |
| 2     | **Cuisine santé et régimes**     | `"vegan", "gluten", "healthy", "fresh", "vegetarian", "organic"`   |
| 3     | **Famille et groupes**           | `"family", "kids", "friendly", "casual", "group", "homey"`         |
| 4     | **Cuisine locale et raffinée**   | `"gourmet", "chef", "local", "authentic", "seasonal", "refined"`   |
| 5     | **Ambiance internationale / bar**| `"cocktails", "music", "dance", "international", "nightlife", "lounge"` |

---

Ces thèmes permettent d’interpréter la sémantique de chaque segment automatiquement identifié par LDA. Ils reflètent des profils distincts d’établissements : des restaurants gastronomiques traditionnels à la restauration rapide, en passant par les bars lounge ou les restaurants familiaux.

> **Pourquoi avons-nous retenu 6 topics ?**

Après avoir expérimenté plusieurs valeurs de K (nombre de sujets), nous avons combiné deux approches complémentaires :
- Une **évaluation quantitative** à l’aide de la métrique de cohérence `c_v`, qui donne une première indication sur la qualité des regroupements thématiques.
- Une **analyse qualitative manuelle**, fondée sur l’inspection des mots-clés dominants et la détection de redondances entre sujets.

####  Courbe de cohérence (score `c_v`) selon le nombre de topics

Afin de visualiser l’évolution de la cohérence en fonction du nombre de sujets, nous avons tracé la courbe suivante :

![Courbe de cohérence LDA](rapport/opti_topic.png)

> *Figure : Score de cohérence `c_v` en fonction du nombre de topics (K). Le point K=6 correspond au modèle retenu.*


La courbe montre une **tendance générale à la hausse** du score de cohérence à mesure que le nombre de topics augmente, mais cette progression est **irrégulière**. Plusieurs pics locaux apparaissent (notamment à K=5, K=12 ou K=17), tandis que d'autres valeurs intermédiaires montrent un **recul temporaire de la cohérence**. Cela suggère que le modèle gagne en complexité sans nécessairement améliorer la qualité des regroupements thématiques.

Nous avons donc combiné cette évaluation automatique avec une **analyse manuelle** des sujets générés. En explorant les topics obtenus pour différentes valeurs de K, nous avons constaté qu’au-delà de **6 sujets**, des **doublons thématiques** apparaissaient régulièrement (ex. : deux groupes sur la gastronomie ou le vin), rendant l’interprétation moins claire.

Afin d’obtenir des thèmes distincts et compréhensibles, nous avons utilisé **ChatGPT** pour interpréter les mots-clés les plus représentatifs de chaque topic. Cela nous a permis d’identifier à K=6 une **structure thématique claire**, avec des segments différenciés et cohérents avec la réalité des profils restaurants (gastronomie, famille, rapide, bar, santé, etc.).

Ce choix nous a permis d’allier **lisibilité**, **interprétabilité** et **richesse thématique**, tout en évitant la surcharge cognitive d’un trop grand nombre de sujets. Ces informations ont été intégrées dans notre **dashboard interactif (Task 3)**, qui permet d’explorer dynamiquement les thèmes identifiés et leur répartition géographique en France.


---




## 2. Task 2.1 – Classification supervisée

### 2.1 Objectif de la tâche

L'objectif de cette tâche est de construire un modèle de classification capable de prédire si un restaurant sera **bien noté ou non** à partir de ses descripteurs textuels (`keywords`). Pour cela, nous avons formulé une tâche de **classification binaire** en utilisant la variable `avg_rating`.

Nous avons défini la variable cible (`target`) de la manière suivante :
- **Classe 1 (positif)** : si `avg_rating >= 4.0`  
- **Classe 0 (négatif)** : si `avg_rating < 4.0`

Cette formulation reflète un cas d’usage réaliste : recommander ou filtrer automatiquement des établissements de qualité à partir de leur profil descriptif.

### 2.2 Méthodologie

Nous avons suivi les étapes classiques d’un pipeline de classification supervisée :

1. **Préparation des données**
   - Texte vectorisé selon 4 méthodes : TF-IDF, BoW, Word2Vec, Doc2Vec
   - Réduction de dimension via `SelectKBest(chi2)` ou `PCA` selon les cas
   - Normalisation automatique via les modèles

2. **Division en ensembles d’apprentissage et de test**
   - Utilisation de `train_test_split` avec un ratio 80% / 20%
   - `random_state=42` pour reproductibilité
   - Strate selon la variable cible pour conserver l'équilibre

3. **Évaluation par métriques standards**
   - Accuracy, F1-score (macro et par classe)
   - Matrice de confusion
   - Aire sous la courbe ROC (AUC)

### 2.3 Modèles testés

Nous avons testé plusieurs algorithmes de classification couvrant une diversité d’approches (linéaires, probabilistes, non linéaires, réseaux de neurones), afin d’identifier ceux les plus adaptés à notre tâche de prédiction binaire.

La **régression logistique** a été notre point de départ en tant que modèle de référence simple, linéaire et interprétable. Elle a donné de bons résultats sur les vecteurs creux comme TF-IDF, mais sa capacité à modéliser des relations complexes reste limitée, ce qui peut pénaliser ses performances sur des données moins linéaires.

Le **Random Forest** s’est montré particulièrement robuste : en combinant plusieurs arbres de décision, il capture naturellement des interactions entre variables, même bruitées. Il tolère bien les données peu normalisées, et a bien fonctionné avec les vecteurs denses (Word2Vec). En revanche, il est moins rapide à entraîner, consomme plus de mémoire, et son fonctionnement interne est plus difficile à interpréter.

Nous avons également testé un **classifieur à base de gradient boosting**, connu pour ses performances sur des jeux de données structurés. Il a offert de bons résultats, mais au prix d’un **temps d'entraînement élevé**, notamment avec des représentations textuelles à forte dimension.

Le **LinearSVC** (Support Vector Classifier linéaire) s’est imposé comme l’un des meilleurs modèles sur les vecteurs TF-IDF. Très efficace en haute dimension, il maximise la marge entre classes, ce qui améliore la généralisation. Il est rapide, robuste, et moins sensible aux outliers que d’autres modèles linéaires. Son principal défaut reste qu’il ne produit pas directement de probabilités, ce qui peut limiter son usage dans des applications nécessitant des scores calibrés.

Le **Naive Bayes**, rapide et facile à entraîner, s’est montré compétitif avec BoW et TF-IDF. Cependant, ses hypothèses d’indépendance entre les mots sont trop fortes, et sa performance chute rapidement avec des vecteurs denses ou en présence de corrélations entre termes.

Le **K-Nearest Neighbors** (KNN), quant à lui, offre une approche intuitive : un point est classé selon ses voisins les plus proches. Il fonctionne bien sur des vecteurs compacts comme Word2Vec, mais il est lent à l’inférence (car le calcul des distances se fait en temps réel) et sensible au bruit ou aux données déséquilibrées.

Enfin, le **Multi-Layer Perceptron (MLP)** représente la solution la plus expressive, capable d’apprendre des non-linéarités complexes. Il a très bien fonctionné avec Word2Vec et Doc2Vec, mais il nécessite un ajustement fin des hyperparamètres, un temps d’entraînement important, et est plus sensible au surapprentissage.

En résumé, chaque modèle présente des compromis entre **performance brute**, **vitesse**, **complexité**, et **interprétabilité**, et notre évaluation a permis de retenir ceux les plus adaptés à notre cas d’usage, en particulier **LinearSVC et Random Forest**, selon le type de vecteur utilisé.

---

### 2.4 Réduction de dimension

#### Sélection supervisée (`SelectKBest` avec chi²)
- Appliquée aux vecteurs creux (TF-IDF, BoW)
- Sélection des **300 features les plus discriminantes**
- Gain observé : **réduction du sur-apprentissage** et **temps d’entraînement réduit**

#### Extraction non supervisée (`PCA`)
- Appliquée aux vecteurs denses (Word2Vec, Doc2Vec)
- Réduction à **50 composantes principales**
- Permet d’extraire la structure latente des documents

---

### 2.5 Résultats comparatifs

Nous avons comparé chaque combinaison **[vecteur + modèle]**. Les résultats sont synthétisés ci-dessous.

### 2.5 Résultats et comparaison des modèles (avec TF-IDF)

Nous présentons ici les performances des modèles testés avec la vectorisation **TF-IDF**, qui s’est révélée être la plus efficace globalement.

#### 📊 Résultats globaux (TF-IDF)

| Modèle               | Accuracy | F1_class_0 | F1_macro | AUC     |
|----------------------|----------|------------|----------|---------|
| **Linear SVC**       | 0.834    | **0.370**  | **0.637**| —       |
| Logistic Regression  | 0.801    | 0.329      | 0.606    | 0.759   |
| MLP (Neural Net)     | 0.884    | 0.241      | 0.589    | 0.640   |
| Random Forest        | 0.879    | 0.116      | 0.525    | 0.598   |
| Gradient Boosting    | 0.893    | 0.069      | 0.506    | 0.667   |
| KNN                  | 0.889    | 0.045      | 0.493    | 0.587   |
| **Naive Bayes**      | **0.897**| 0.000      | 0.473    | **0.807**|

> *Tableau : Résultats des principaux modèles entraînés sur les vecteurs TF-IDF. L'accent est mis sur la F1-score classe 0, essentielle pour prédire les restaurants mal notés.*

---

#### Courbe ROC – TF-IDF

![Courbe ROC TF-IDF](rapport/tf_courbe_auc.png)


> *Figure : Comparaison des courbes ROC pour les modèles TF-IDF. Naive Bayes affiche l’AUC la plus élevée mais présente un fort biais de prédiction.*

---

####  Matrices de confusion

##### 🔹 Naive Bayes (TF-IDF)

![Confusion Matrix – Naive Bayes](rapport/tf_naive.png)

> Le modèle **ne prédit jamais la classe 0** : tous les restaurants sont classés comme "bien notés", ce qui explique sa F1_score = 0 sur cette classe, malgré une AUC élevée.

##### 🔹 Random Forest (TF-IDF)

![Confusion Matrix – Random Forest](rapport/tf_random.png)

> L’arbre aléatoire prédit la classe 0, mais avec beaucoup de **faux positifs**. Son F1_class_0 reste faible (0.116), malgré une meilleure balance que Naive Bayes.

### 2.5 Résultats et comparaison des modèles (avec BoW)

Nous présentons ici les performances des modèles testés avec la vectorisation **Bag of Words (BoW)**. Bien que plus simple que TF-IDF, cette méthode s’est révélée compétitive dans plusieurs cas, notamment avec Naive Bayes.

####  Résultats globaux (BoW)

| Modèle               | Accuracy | F1_class_0 | F1_macro | AUC     |
|----------------------|----------|------------|----------|---------|
| **Naive Bayes**      | **0.910**| **0.393**  | **0.672**| **0.842**|
| Logistic Regression  | 0.788    | 0.353      | 0.613    | 0.767   |
| Linear SVC           | 0.806    | 0.329      | 0.608    | —       |
| MLP (Neural Net)     | 0.871    | 0.210      | 0.570    | 0.654   |
| Random Forest        | 0.868    | 0.074      | 0.501    | 0.651   |
| Gradient Boosting    | 0.893    | 0.024      | 0.483    | 0.671   |
| KNN                  | 0.892    | 0.000      | 0.471    | 0.532   |

> *Tableau : Performances des modèles avec BoW. Naive Bayes ressort comme le modèle dominant sur toutes les métriques clés, y compris la classe minoritaire.*

---

####  Courbe ROC – BoW

![Courbe ROC BoW](rapport/bow_auc.png)

> *Figure : Comparaison des courbes ROC pour les modèles BoW. Naive Bayes montre la meilleure séparation (AUC = 0.84), suivi de Logistic Regression.*

---

####  Matrices de confusion

##### 🔹 Naive Bayes (BoW)

![Confusion Matrix – Naive Bayes](rapport/bow_naive.png)

> Contrairement à sa version TF-IDF, Naive Bayes ici **réussit à prédire la classe 0** (22 cas sur 78), tout en gardant une bonne précision globale.

##### 🔹 Random Forest (BoW)

![Confusion Matrix – Random Forest](rapport/bow_randomm.png)

> Random Forest détecte très peu de cas négatifs (classe 0), avec une forte **asymétrie de prédiction**. Cela confirme son **F1_class_0 très faible (0.074)** malgré une bonne accuracy globale.

---

###  Analyse comparative des modèles avec BoW

Les performances observées avec la vectorisation BoW soulignent plusieurs éléments importants :

- **Naive Bayes** atteint la **meilleure accuracy (91%)**, **meilleure F1_class_0 (0.393)**, **meilleure F1_macro (0.672)** et **meilleure AUC (0.84)**. Il est donc, ici, **le modèle le plus performant**, et de loin, notamment grâce à la simplicité de la représentation BoW qu’il exploite parfaitement.

- **Logistic Regression** confirme sa robustesse avec une bonne **F1_class_0 (0.353)** et un **AUC élevé (0.77)**. C’est un **excellent compromis** entre complexité modérée et performance.

- **Linear SVC**, sans AUC disponible, reste compétitif mais **moins efficace que sur TF-IDF**.

- Les modèles plus complexes comme **Random Forest**, **Gradient Boosting** ou **MLP** sont ici **surclassés par des modèles plus simples**. Leurs performances sur la classe minoritaire restent décevantes (F1_class_0 < 0.1 dans certains cas).

---

###  Conclusion sur les résultats BoW

Avec la vectorisation BoW, c’est **Naive Bayes** qui s’impose comme le **modèle le plus performant**, contrairement à TF-IDF où il échouait à détecter la classe minoritaire. Cela confirme que le **choix du modèle dépend étroitement du type de représentation vectorielle**.

BoW, bien que basique, s’avère très efficace couplé à un modèle probabiliste adapté à ses hypothèses, comme Naive Bayes. En revanche, pour les approches plus complexes ou non-linéaires, cette représentation semble trop simpliste pour exprimer correctement les nuances textuelles nécessaires à une classification fine.

###  2.5 Résultats et comparaison des modèles (avec Word2Vec)

Nous présentons ici les performances des modèles testés avec la vectorisation **Word2Vec**, qui encode chaque mot comme un vecteur dense appris à partir du contexte. Pour chaque restaurant, nous avons pris la moyenne des vecteurs mots présents.

####  Résultats globaux (Word2Vec)

| Modèle               | Accuracy | F1_class_0 | F1_macro | AUC     |
|----------------------|----------|------------|----------|---------|
| Linear SVC           | 0.585    | 0.245      | 0.479    | —       |
| Logistic Regression  | 0.594    | 0.238      | 0.481    | 0.607   |
| KNN                  | 0.892    | 0.089      | 0.516    | 0.563   |
| Gradient Boosting    | 0.899    | 0.072      | 0.509    | 0.657   |
| MLP (Neural Net)     | 0.899    | 0.025      | 0.486    | 0.663   |
| Random Forest        | 0.897    | 0.000      | 0.473    | 0.646   |

> *Tableau : Résultats obtenus avec Word2Vec. Les modèles semblent obtenir une bonne accuracy, mais leur capacité à détecter la classe minoritaire reste limitée.*

---

####  Courbe ROC – Word2Vec

![Courbe ROC Word2Vec](rapport/w_courbe_auc.png)

> *Figure : Comparaison des courbes ROC pour les modèles Word2Vec. Les meilleurs AUC sont obtenus par MLP et Gradient Boosting (~0.66).*

---

####  Matrices de confusion

##### 🔹 Gradient Boosting (Word2Vec)

![Confusion Matrix – Gradient Boosting](rapport/w_gb.png)

> Le modèle parvient à identifier une petite partie de la classe 0 (3 prédictions correctes), mais reste fortement biaisé vers la classe majoritaire.

##### 🔹 MLP (Word2Vec)

![Confusion Matrix – MLP](rapport/w_mlp.png)

> Le réseau de neurones ne prédit **pratiquement jamais** la classe 0, malgré un AUC raisonnable. Cela explique sa F1_class_0 très faible (0.025).

---

###  Analyse comparative des modèles avec Word2Vec

L’utilisation de **Word2Vec** n’a pas apporté les gains espérés par rapport à TF-IDF ou BoW. Plusieurs constats peuvent être faits :

- Les **modèles linéaires** comme **Linear SVC** et **Logistic Regression** affichent des scores modestes (accuracy ~0.59), mais avec une **F1_class_0 autour de 0.24**, ce qui reste acceptable pour cette tâche déséquilibrée.

- Les **modèles complexes (MLP, GB, RF)**, bien que très précis globalement (accuracy ~0.89), **échouent à identifier correctement les mauvais restaurants (classe 0)**. Leurs matrices de confusion le confirment.

- **MLP**, par exemple, affiche un AUC correct (0.66) mais **ne prédit la classe 0 qu’une seule fois**, le rendant peu utile dans une logique de filtrage/recommandation équilibrée.

- **KNN** et **Random Forest** obtiennent des résultats similaires : **accuracy forte**, mais **F1_class_0 < 0.1**, donc très faible sensibilité aux mauvais restaurants.

---

###  Conclusion sur les résultats Word2Vec

Word2Vec, malgré sa richesse sémantique théorique, s’est révélé **moins performant que prévu** dans ce contexte. La principale raison est la **structure très courte et catégorielle des données** (`keywords`), qui ne bénéficie pas pleinement des relations contextuelles apprises par Word2Vec.

Les performances globales sont **fortement biaisées vers la classe majoritaire**, ce qui entraîne une **perte d’utilité** pour un système de recommandation qui doit justement pouvoir **discriminer les établissements moins appréciés**.

Ainsi, malgré un gain potentiel en expressivité vectorielle, **Word2Vec n’a pas surpassé les méthodes classiques**, et reste secondaire dans notre pipeline.

###  2.6 Résultats et comparaison des modèles (avec Doc2Vec)

Nous présentons ici les performances obtenues avec **Doc2Vec**, une méthode qui apprend un vecteur dense propre à chaque document (restaurant), au lieu d’agréger des vecteurs mots.

####  Résultats globaux (Doc2Vec)

| Modèle               | Accuracy | F1_class_0 | F1_macro | AUC     |
|----------------------|----------|------------|----------|---------|
| Linear SVC           | 0.635    | 0.277      | 0.516    | —       |
| Logistic Regression  | 0.640    | 0.268      | 0.514    | 0.711   |
| MLP (Neural Net)     | 0.884    | 0.170      | 0.553    | 0.650   |
| KNN                  | 0.890    | 0.162      | 0.552    | 0.572   |
| Gradient Boosting    | 0.895    | 0.070      | 0.506    | 0.643   |
| Random Forest        | 0.897    | 0.000      | 0.473    | 0.653   |

> *Tableau : Résultats des modèles avec vecteurs Doc2Vec. Bien que l’accuracy soit souvent élevée, la capacité à prédire la classe 0 reste globalement limitée.*

---

####  Courbe ROC – Doc2Vec

![Courbe ROC Doc2Vec](rapport/d_auc.png)

> *Figure : Comparaison des courbes ROC pour Doc2Vec. Le meilleur AUC est obtenu par la régression logistique (0.71), devant MLP et Gradient Boosting.*

---

####  Matrices de confusion

##### 🔹 Logistic Regression (Doc2Vec)

![Confusion Matrix – Logistic Regression](rapport/d_lin.png)

> Ce modèle arrive à équilibrer relativement les classes. Il détecte correctement une partie non négligeable des classes 0 (50 sur 78), mais rate encore 245 exemples.

##### 🔹 KNN (Doc2Vec)

![Confusion Matrix – KNN](rapport/d_knn.png)

> Bien qu’il offre une bonne accuracy, KNN tend à prédire la classe majoritaire. La classe 0 n’est détectée que dans 8 cas sur 78.

---

### 🔍 Analyse comparative des modèles avec Doc2Vec

L’approche **Doc2Vec** propose une modélisation plus riche, en capturant le sens global d’un document plutôt que le cumul des mots. Cela dit, dans notre contexte — des documents très courts composés uniquement de `keywords` — cette richesse reste sous-exploitée.

- **Linear SVC** et **Logistic Regression** affichent des performances relativement stables. Logistic Regression s'en tire le mieux avec un **AUC de 0.71**, et un **F1_class_0 autour de 0.27**, ce qui reste le meilleur score de cette série.

- Les modèles non-linéaires comme **MLP** et **KNN** atteignent une **accuracy élevée (~0.89)**, mais souffrent encore d’un **déséquilibre de classes**, avec une **sous-détection forte des mauvais restaurants**.

- Les **Random Forest** et **Gradient Boosting** affichent une absence quasi-totale de prédiction pour la classe 0. Malgré une AUC correcte (~0.65), ils **échouent totalement** sur cette classe.

> À noter que **la meilleure combinaison ici reste Doc2Vec + Logistic Regression**, mais elle ne surpasse pas TF-IDF en termes de F1_class_0.

---

###  Conclusion sur les résultats Doc2Vec

L’approche Doc2Vec s’avère **trop sophistiquée pour un corpus composé de chaînes courtes et peu ambiguës** comme les `keywords`. Son avantage sur des documents longs ne se traduit pas ici par de meilleures performances.

Elle souffre du même problème que Word2Vec : **les modèles interprètent mal la classe 0**, ce qui les rend inadaptés à des systèmes de recommandation équilibrés. Enfin, son **coût de calcul est significatif**, pour un **gain limité**, ce qui renforce notre préférence pour **TF-IDF** dans ce cas d’usage.



###  Analyse comparative

La comparaison des modèles fait apparaître plusieurs **compromis** :

- **Naive Bayes** obtient le **meilleur score AUC (0.81)**, et la **meilleure accuracy (0.897)**, mais il est **incapable de prédire correctement les mauvais restaurants** (classe 0), ce qui **le rend inutilisable** pour une application de recommandation équilibrée.
  
- **Linear SVC** a le **meilleur F1_class_0 (0.370)** et le **meilleur F1_macro (0.637)**, ce qui indique une **meilleure capacité à distinguer les deux classes** — y compris les minoritaires. Même sans score de probabilité (pas d’AUC), son **équilibre entre précision et robustesse** en fait le **modèle le plus fiable** pour notre tâche.

- **Logistic Regression** est un bon compromis : des scores corrects partout, une AUC de 0.76 et un F1_class_0 satisfaisant (0.33), ce qui le rend viable si une probabilité explicite est nécessaire.

- **Random Forest**, bien qu’intéressant sur les features complexes, souffre ici d’un **F1_class_0 trop faible (0.116)**, ce qui diminue sa pertinence dans notre cas.

- Les modèles comme **KNN** ou **Gradient Boosting** ont un bon score global, mais **échouent sur la classe minoritaire** (F1 < 0.1), les rendant peu adaptés à une tâche déséquilibrée.


---

###  Conclusion générale sur la comparaison des modèles et vectorisations

L’analyse croisée de l’ensemble des résultats obtenus avec les différentes méthodes de vectorisation (TF-IDF, BoW, Word2Vec, Doc2Vec) et modèles de classification supervisée permet de tirer plusieurs enseignements concrets.

####  Performances globales

Les meilleurs compromis entre précision globale (accuracy) et capacité à identifier correctement les restaurants **mal notés** (F1_class_0) ont été atteints avec :

- **TF-IDF + LinearSVC** : F1_class_0 = 0.37, F1_macro = 0.637  
- **BoW + Naive Bayes** : F1_class_0 = 0.392, F1_macro = 0.672, AUC = 0.84  
- **TF-IDF + Logistic Regression** : F1_class_0 = 0.329, F1_macro = 0.606  

Les scores les plus élevés d’**AUC (area under ROC curve)** sont observés avec **BoW + Naive Bayes** et **TF-IDF + Logistic Regression**, indiquant une bonne capacité à distinguer les classes au niveau probabiliste.

####  Pourquoi certaines AUC sont à `NaN` ?

Les valeurs `NaN` dans la colonne **AUC** (Area Under the Curve) proviennent du fait que certains modèles (notamment **LinearSVC**) ne disposent pas de méthode `predict_proba()` dans leur implémentation de Scikit-learn. Cette méthode est nécessaire pour calculer l’AUC, car elle fournit des **probabilités de classe**. En son absence, seule la prédiction binaire (`predict()`) est disponible, ce qui empêche la génération de la courbe ROC et donc le calcul de l’aire sous celle-ci.

Cela explique pourquoi **LinearSVC** apparaît avec un `NaN` dans la colonne AUC, malgré des performances tout à fait compétitives en accuracy et F1-score.

####  Comparaison des méthodes de vectorisation

- **TF-IDF** s’impose comme la méthode la plus **fiable, stable et efficace**, notamment avec des modèles linéaires. Elle combine une bonne précision globale avec une bonne sensibilité à la classe minoritaire.
- **BoW**, bien que simpliste, donne d’excellents résultats avec **Naive Bayes**, prouvant qu’un modèle simple peut suffire dans un contexte bien préparé.
- **Word2Vec** et **Doc2Vec** montrent des performances globales correctes en accuracy, mais souvent faibles en **F1_class_0**, ce qui montre leur difficulté à prédire les restaurants mal notés. De plus, leur coût computationnel est élevé.

---

###  Conclusion finale

En résumé, **la combinaison TF-IDF + LinearSVC** est le meilleur choix dans notre cas d’usage : elle offre un très bon **F1 sur la classe 0**, une **mise en œuvre rapide**, une **interprétabilité correcte**, et une bonne **généralisation**.

Les représentations vectorielles denses (Word2Vec, Doc2Vec), bien que prometteuses théoriquement, ne sont pas adaptées à un corpus aussi court et structuré que le nôtre. Le **rapport coût / performance** ne justifie pas leur usage ici.

Enfin, **l’utilisation de plusieurs métriques** (accuracy, F1_class_0, AUC) a permis d’avoir une vision complète et équilibrée des performances, en particulier dans un contexte déséquilibré où la **classe minoritaire (restaurants mal notés)** est la plus difficile à détecter, mais aussi la plus utile pour notre système de recommandation.





## 3. Task 3 – Tableaux de bord interactifs avec Dash

###  Objectif général

Dans une optique de lisibilité, d’interprétabilité et d'interaction, nous avons développé **deux tableaux de bord interactifs** avec la bibliothèque **Dash (Plotly)**. Ces outils permettent :

- D'explorer les **sujets thématiques extraits par LDA** de manière géographique et interprétable
- De naviguer dans les **résultats de classification supervisée**, en testant dynamiquement différents modèles et méthodes de vectorisation

>  *Pour concevoir ces interfaces, nous avons utilisé ChatGPT pour optimiser les callbacks, structurer les interactions et améliorer l’ergonomie.*

---

###  Dashboard LDA – Visualisation des sujets

Ce tableau de bord permet d’explorer les **sujets latents (topics)** identifiés par LDA, en filtrant par **ville** et en visualisant la répartition des thèmes sur une **carte interactive**.

#### Fonctionnalités principales :
- **Dropdown de ville** : filtrer dynamiquement les restaurants par localité
- **Histogramme des sujets dominants** : voir la répartition des topics (0 à 5)
- **Carte Mapbox** : localisation des restaurants colorés selon leur sujet
- **Affichage des mots-clés d’un topic** : aide à l’interprétation sémantique

#### Ce qu’on peut y voir :
- Les thématiques culinaires les plus présentes dans une ville
- Les zones géographiques associées à chaque type de restaurant
- Les descripteurs caractéristiques de chaque sujet (ex : "wine bar", "cheap", "gluten free")

#### Capture d’écran (à insérer) :
![Dashboard LDA – Histogramme et carte](data/d1p1.png)
![Dashboard LDA – Histogramme et carte](data/d1p2.png)
![Dashboard LDA – Histogramme et carte](data/d1_p3.png)


---

###  Dashboard de recommandation – Modèles de prédiction

Ce second tableau de bord permet de visualiser les **restaurants prédits comme bien notés** à partir de leurs mots-clés, en fonction du **modèle** et de la **méthode de vectorisation** sélectionnés.

#### Fonctionnalités principales :
- **Dropdown de vectorisation** : TF-IDF, BoW, Word2Vec, Doc2Vec
- **Dropdown de modèle** : Logistic Regression, Random Forest, SVC, MLP, etc.
- **Carte des prédictions** : restaurants prédits comme "bons" affichés avec leur score
- **Top 10 recommandations** : tableau des restaurants les mieux classés
- **Top 10 pires restaurants** : possibilité d'afficher également les moins bien notés

#### Ce qu’on peut y voir :
- L’impact des choix de modèle/vectorisation sur les résultats de recommandation
- Les meilleurs et pires restaurants selon chaque combinaison
- Des détails pour chaque établissement (ville, note, mots-clés, score)

#### Capture d’écran (à insérer) :
![Dashboard Recommandation – Carte](chemin/vers/image_reco_dashboard.png)

---

###  Intérêt global

Ces deux dashboards rendent le projet :
- **Accessible** aux non-experts
- **Explorable visuellement** en temps réel
- **Facilement réutilisable** dans des cas pratiques de recommandation

Ils peuvent être adaptés pour des systèmes de filtrage, de recherche ou de notation en ligne.

> Une courte vidéo de démonstration est disponible en annexe pour illustrer les interactions proposées.


# Conclusion générale

Ce projet a permis de mettre en œuvre une chaîne complète d’analyse automatique appliquée à des données textuelles, depuis le prétraitement jusqu’à la visualisation interactive. En partant d’un jeu de données extrait de TripAdvisor, nous avons concentré notre analyse sur les restaurants situés en France. À travers un ensemble de mots-clés décrivant les caractéristiques des établissements, nous avons cherché à extraire des informations thématiques pertinentes, puis à prédire la qualité perçue de chaque restaurant selon sa note moyenne.

La première étape a consisté à nettoyer, structurer et transformer les données textuelles via différentes méthodes de vectorisation : TF-IDF, Bag of Words, Word2Vec et Doc2Vec. Nous avons pu constater que les méthodes simples comme TF-IDF, lorsqu’elles sont bien adaptées à la nature des données (ici, des mots-clés courts et synthétiques), restent particulièrement efficaces. En particulier, la combinaison TF-IDF avec un modèle LinearSVC s’est révélée être la plus stable et performante pour prédire les établissements bien notés, avec une F1-score satisfaisante sur la classe minoritaire.

En parallèle, nous avons utilisé des méthodes non supervisées comme LDA (Latent Dirichlet Allocation) pour identifier des sujets latents dans le corpus. L’algorithme a permis d’extraire des thématiques distinctes (restaurants familiaux, bars, gastronomiques, végétariens…) que nous avons interprétées à partir des mots-clés dominants. Le choix du nombre optimal de sujets a été guidé à la fois par une courbe de cohérence (c_v) et une analyse qualitative assistée par ChatGPT, nous permettant d’équilibrer lisibilité et richesse sémantique.

Enfin, afin de rendre les résultats accessibles à un public non expert, nous avons développé deux tableaux de bord interactifs à l’aide de Dash. Le premier permet d’explorer géographiquement les thématiques LDA à travers une carte et des filtres par ville. Le second tableau de bord sert de système de recommandation visuel, où l’utilisateur peut tester différents modèles de prédiction et explorer les restaurants recommandés (ou à éviter), selon les scores et vecteurs associés.

Ce projet a donc été l’occasion d’appliquer de manière concrète les principaux outils du traitement automatique du langage naturel et de l’apprentissage supervisé. Il démontre qu’un corpus même limité (ici, des mots-clés) peut donner lieu à des analyses riches, des modèles performants et des interfaces intelligentes. Plus largement, ce travail illustre l’intérêt de croiser rigueur technique, visualisation interactive et aide à l’interprétation, pour transformer un ensemble de données textuelles en outil d’exploration et de décision.